# Convolution Image Net

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np
import os
import shutil
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
from PIL import Image
from utils.nn_graph import simple_layer
from utils.data import init_dir
from utils.nn_visualization import init_embedding_projector, init_embedding_data
from utils.nn_visualization import get_sprite_img, get_label_class_names

## Load data

In [3]:
data = input_data.read_data_sets('/data/fashion/', one_hot=True)
img_shape = (28, 28)
class_id2class_name_mapping = {
    0: 'T-shirt/top',
    1: 'Trouser',
    2: 'Pullover',
    3: 'Dress',
    4: 'Coat',
    5: 'Sandal',
    6: 'Shirt',
    7: 'Sneaker',
    8: 'Bag',
    9: 'Ankle boot'}

Extracting /data/fashion/train-images-idx3-ubyte.gz
Extracting /data/fashion/train-labels-idx1-ubyte.gz
Extracting /data/fashion/t10k-images-idx3-ubyte.gz
Extracting /data/fashion/t10k-labels-idx1-ubyte.gz


## Build Conv Net Graph

In [4]:
from utils.nn_visualization import conv33132_summaries, conv55124_summary, variable_summaries

In [5]:
graph = tf.Graph()
with graph.as_default():
    with tf.name_scope('conv_image_net_inputs'):
        images = tf.placeholder(tf.float32, shape=[None, 784], name='images')
        labels = tf.placeholder(tf.float32, shape=[None, 10], name='labels')
        keep_dropout_prob = tf.placeholder(tf.float32, name='keep_dropout_prob')
        is_training = tf.placeholder(tf.bool, name='is_training')
            
    with tf.name_scope('image_reshape'):        
        images_reshaped = tf.reshape(images, [-1, 28, 28, 1])

    with tf.variable_scope('conv_layer_1'):
        w_conv_1 = tf.get_variable('w_conv_1', [3, 3, 1, 32], initializer=tf.contrib.layers.variance_scaling_initializer())
        b_conv_1 = tf.get_variable('b_conv_1', initializer=tf.constant_initializer(0), shape=[32])
        conv33132_summaries('w_conv_1_summary', w_conv_1)

        conv_layer_1 = tf.nn.conv2d(images_reshaped, w_conv_1, strides=[1, 1, 1, 1], padding='SAME')
        # conv_layer_1 = tf.contrib.layers.batch_norm(conv_layer_1, is_training=is_training)
        conv_layer_1 = conv_layer_1 + b_conv_1
        conv_layer_1 = tf.nn.relu(conv_layer_1)
        conv_layer_1 = tf.nn.max_pool(conv_layer_1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')   

    with tf.variable_scope('conv_layer_2'):
        w_conv_2 = tf.get_variable('w_conv_2', [3, 3, 32, 64], initializer=tf.contrib.layers.variance_scaling_initializer())
        b_conv_2 = tf.get_variable('b_conv_2', initializer=tf.constant_initializer(0), shape=[64])
        
        conv_layer_2 = tf.nn.conv2d(conv_layer_1, w_conv_2, strides=[1, 1, 1, 1], padding='SAME')
        # conv_layer_2 = tf.contrib.layers.batch_norm(conv_layer_2, is_training=is_training)
        conv_layer_2 = conv_layer_2 + b_conv_2
        conv_layer_2 = tf.nn.relu(conv_layer_2)
        conv_layer_2 = tf.nn.max_pool(conv_layer_2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    with tf.variable_scope('feed_forward_layer_1'):
        ff_layer_1 = tf.reshape(conv_layer_2, [-1, 7*7*64])
        ff_layer_1 = simple_layer('ff_1', ff_layer_1, shape=[7*7*64, 10], activation='linear')
        raw_prediction = tf.nn.dropout(ff_layer_1, keep_dropout_prob)
    
    with tf.name_scope('prediction'):
        prediction = tf.nn.softmax(raw_prediction)
    
    with tf.variable_scope("embedding_visualization"):
        embedding = tf.Variable(tf.zeros([5000, 10]), name='valid_embedding')
        embedding_assignment = embedding.assign(raw_prediction)

    with tf.name_scope('loss'):
        cross_entropy_vector = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=raw_prediction)
        print('Debug [cross_etropy_vector]:', cross_entropy_vector.get_shape())
        loss = tf.reduce_mean(cross_entropy_vector)
        variable_summaries('loss_summary', cross_entropy_vector)
        
    with tf.name_scope('training'):
        update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
        with tf.control_dependencies(update_ops):
            train_step  = tf.train.AdamOptimizer(1e-3).minimize(loss)

    with tf.name_scope('accuracy'):
        correct_prediction = tf.equal(tf.argmax(prediction,1), tf.argmax(labels,1))
        correct_prediction = tf.cast(correct_prediction, tf.float32)
        accuracy = tf.reduce_mean(correct_prediction)
        variable_summaries('accuracy_summary', correct_prediction)
    
    initialize_vars = tf.global_variables_initializer()
    merge_summaries = tf.summary.merge_all()

Debug [cross_etropy_vector]: (?,)


## Init Model Logging

In [6]:
from utils.data import init_model_logging
base_dir = '/tensorboard_summaries/conv_image_net/'
exp_name = 'experiment_final'

logging_meta = init_model_logging(base_dir, exp_name, graph=graph, remove_existing=True)

## Add Embedding Projection

In [7]:
sprite_img = get_sprite_img(data.validation.images, img_shape)
label_names = get_label_class_names(data.validation.labels, class_id2class_name_mapping)

init_embedding_data(logging_meta['valid_writer_dir'], sprite_img, label_names)
init_embedding_projector(logging_meta['valid_writer'], embedding, img_shape)

## Run Conv Net

In [8]:
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth = True
model_path = logging_meta['model_path']

validation_feed_dict = {
    images: data.validation.images, 
    labels: data.validation.labels,
    keep_dropout_prob: 1., 
    is_training: False}

session = tf.Session(graph=graph, config=config)
session.run(initialize_vars)
# logging_meta['saver'].restore(session, '/tensorboard_summaries/conv_image_net/experiment_1/valid/model.ckpt-4900')

for iteration in range(5000):
    ##################
    # Training phase #
    ##################
    _images, _labels = data.train.next_batch(100)
    feed_dict = {images: _images, labels: _labels, keep_dropout_prob: 0.5, is_training: True}
    _ = session.run([train_step], feed_dict=feed_dict)

    if iteration % 10 == 0:
        feed_dict={images: _images, labels: _labels, keep_dropout_prob: 1., is_training: False}
        _summary, _accuracy, _loss = session.run([merge_summaries, accuracy, loss],feed_dict=feed_dict)

        logging_meta['train_writer'].add_summary(_summary, iteration)
        print("Train Iteration {}: loss {}, accuracy {}".format(iteration, _loss, _accuracy))

    ####################
    # Validation phase #
    ####################
    if iteration % 100 == 0:
        fetches = [embedding_assignment, merge_summaries, accuracy, loss]
        _, _summary, _accuracy, _loss = session.run(fetches, validation_feed_dict)

        logging_meta['valid_writer'].add_summary(_summary, iteration)
        logging_meta['saver'].save(session, model_path, iteration)
        print("= Valid Iteration {}: loss {}, accuracy {} =".format(iteration, _loss, _accuracy))

_prediction, = session.run([prediction], feed_dict=validation_feed_dict)

Train Iteration 0: loss 2.182243585586548, accuracy 0.30000001192092896
= Valid Iteration 0: loss 2.289867639541626, accuracy 0.20280000567436218 =
Train Iteration 10: loss 1.0675846338272095, accuracy 0.7200000286102295
Train Iteration 20: loss 1.0855540037155151, accuracy 0.5799999833106995
Train Iteration 30: loss 0.9468861222267151, accuracy 0.7599999904632568
Train Iteration 40: loss 0.8134348392486572, accuracy 0.7599999904632568
Train Iteration 50: loss 0.7536531686782837, accuracy 0.7699999809265137
Train Iteration 60: loss 0.7573251128196716, accuracy 0.7099999785423279
Train Iteration 70: loss 0.7410078644752502, accuracy 0.800000011920929
Train Iteration 80: loss 0.704986572265625, accuracy 0.7599999904632568
Train Iteration 90: loss 0.7159327864646912, accuracy 0.75
Train Iteration 100: loss 0.5893170833587646, accuracy 0.8299999833106995
= Valid Iteration 100: loss 0.5888126492500305, accuracy 0.8342000246047974 =
Train Iteration 110: loss 0.6716362237930298, accuracy 0.75

= Valid Iteration 1000: loss 0.34487393498420715, accuracy 0.899399995803833 =
Train Iteration 1010: loss 0.30527955293655396, accuracy 0.9300000071525574
Train Iteration 1020: loss 0.3203602135181427, accuracy 0.9300000071525574
Train Iteration 1030: loss 0.40870895981788635, accuracy 0.8500000238418579
Train Iteration 1040: loss 0.37987425923347473, accuracy 0.8799999952316284
Train Iteration 1050: loss 0.31302210688591003, accuracy 0.9200000166893005
Train Iteration 1060: loss 0.3226400315761566, accuracy 0.9100000262260437
Train Iteration 1070: loss 0.3709760308265686, accuracy 0.8700000047683716
Train Iteration 1080: loss 0.2897009551525116, accuracy 0.9100000262260437
Train Iteration 1090: loss 0.4554344117641449, accuracy 0.8500000238418579
Train Iteration 1100: loss 0.2514658272266388, accuracy 0.9599999785423279
= Valid Iteration 1100: loss 0.3374021649360657, accuracy 0.9031999707221985 =
Train Iteration 1110: loss 0.3289186954498291, accuracy 0.9100000262260437
Train Iterati

Train Iteration 2000: loss 0.21524329483509064, accuracy 0.9300000071525574
= Valid Iteration 2000: loss 0.2843475341796875, accuracy 0.9142000079154968 =
Train Iteration 2010: loss 0.36429354548454285, accuracy 0.8899999856948853
Train Iteration 2020: loss 0.2597625255584717, accuracy 0.9100000262260437
Train Iteration 2030: loss 0.3422115445137024, accuracy 0.8899999856948853
Train Iteration 2040: loss 0.2937374413013458, accuracy 0.9100000262260437
Train Iteration 2050: loss 0.2952677309513092, accuracy 0.8600000143051147
Train Iteration 2060: loss 0.2797790765762329, accuracy 0.9100000262260437
Train Iteration 2070: loss 0.2607831060886383, accuracy 0.9399999976158142
Train Iteration 2080: loss 0.24437494575977325, accuracy 0.9200000166893005
Train Iteration 2090: loss 0.33426231145858765, accuracy 0.9100000262260437
Train Iteration 2100: loss 0.256189227104187, accuracy 0.9300000071525574
= Valid Iteration 2100: loss 0.3036945164203644, accuracy 0.9014000296592712 =
Train Iteratio

Train Iteration 2990: loss 0.24078170955181122, accuracy 0.9300000071525574
Train Iteration 3000: loss 0.22154812514781952, accuracy 0.9399999976158142
= Valid Iteration 3000: loss 0.2739214599132538, accuracy 0.9128000140190125 =
Train Iteration 3010: loss 0.2357863187789917, accuracy 0.949999988079071
Train Iteration 3020: loss 0.239623561501503, accuracy 0.9300000071525574
Train Iteration 3030: loss 0.17081744968891144, accuracy 0.9800000190734863
Train Iteration 3040: loss 0.27929070591926575, accuracy 0.8899999856948853
Train Iteration 3050: loss 0.225957989692688, accuracy 0.9300000071525574
Train Iteration 3060: loss 0.2420121729373932, accuracy 0.8899999856948853
Train Iteration 3070: loss 0.2425592988729477, accuracy 0.8700000047683716
Train Iteration 3080: loss 0.264817476272583, accuracy 0.9300000071525574
Train Iteration 3090: loss 0.1853615939617157, accuracy 0.9700000286102295
Train Iteration 3100: loss 0.2883029878139496, accuracy 0.9200000166893005
= Valid Iteration 310

Train Iteration 3980: loss 0.15645207464694977, accuracy 0.949999988079071
Train Iteration 3990: loss 0.23886485397815704, accuracy 0.9200000166893005
Train Iteration 4000: loss 0.18868397176265717, accuracy 0.949999988079071
= Valid Iteration 4000: loss 0.2535839378833771, accuracy 0.9136000275611877 =
Train Iteration 4010: loss 0.22331103682518005, accuracy 0.9300000071525574
Train Iteration 4020: loss 0.18922770023345947, accuracy 0.9700000286102295
Train Iteration 4030: loss 0.2463078647851944, accuracy 0.8899999856948853
Train Iteration 4040: loss 0.21022620797157288, accuracy 0.949999988079071
Train Iteration 4050: loss 0.2248917520046234, accuracy 0.9300000071525574
Train Iteration 4060: loss 0.30038002133369446, accuracy 0.8799999952316284
Train Iteration 4070: loss 0.1601504683494568, accuracy 0.9300000071525574
Train Iteration 4080: loss 0.12206967175006866, accuracy 0.9900000095367432
Train Iteration 4090: loss 0.1648571789264679, accuracy 0.949999988079071
Train Iteration 4

Train Iteration 4960: loss 0.2391170859336853, accuracy 0.9300000071525574
Train Iteration 4970: loss 0.18440674245357513, accuracy 0.9300000071525574
Train Iteration 4980: loss 0.1452552080154419, accuracy 0.9399999976158142
Train Iteration 4990: loss 0.19584491848945618, accuracy 0.949999988079071


In [ ]:
session.close()

## Results evaluation

In [ ]:
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_accuracy
from utils.results_evaluation import get_false_positives
from utils.results_evaluation import get_info_df
from utils.results_evaluation import get_rec_prec
from utils.results_evaluation import plot_coocurance_matrix
from utils.results_evaluation import plot_examples 

In [ ]:
df = get_info_df(data.validation.labels, _prediction, class_id2class_name_mapping, data.validation.images)

In [ ]:
get_accuracy(df)

In [ ]:
get_accuracy(df, True)

In [ ]:
get_rec_prec(df, class_id2class_name_mapping)

In [ ]:
fp = get_false_positives(df, 'Shirt')

In [ ]:
plot_examples(fp)

In [ ]:
plot_coocurance_matrix(df, use_top3=False, use_log=False)